## Extract

In [19]:
# Install required packages
%pip install openpyxl pandas --quiet
import sqlite3
from datetime import datetime, timedelta

def extract(data_source='C:\\Users\\flags\\End-sem DSA 2040\\DSA-2040_Practical_Exam_PatriciaKiarie781\\Data Warehousing\\ETL_Process\\raw\\Online Retail.xlsx'):
    """
    Extracts data from an Excel file into a pandas DataFrame.
    Handles missing values, data type conversions, and basic data validation.
    
    Args:
        data_source (str): Path to the Excel file containing retail data
        
    Returns:
        pd.DataFrame or None: Cleaned DataFrame if successful, None if extraction fails
    """
    print("\n--- Starting Extraction Phase ---")
    
    try:
        # Read the Excel file
        df_raw = pd.read_excel(data_source)
        print(f"Data extracted from {data_source}. Total rows: {len(df_raw)}")
        
        # Basic data validation
        required_columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 
                          'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
        missing_columns = [col for col in required_columns if col not in df_raw.columns]
        if missing_columns:
            print(f"Error: Missing required columns: {missing_columns}")
            return None
            
        # Handle missing values
        print("\nMissing values before cleaning:")
        print(df_raw.isnull().sum())
        
        # Drop rows with missing CustomerID as it's crucial for the customer dimension
        df_raw.dropna(subset=['CustomerID'], inplace=True)
        
        # Fill missing values for other columns
        df_raw['Description'].fillna('Unknown Product', inplace=True)
        df_raw['Country'].fillna('Unknown', inplace=True)
        
        # Convert data types
        df_raw['CustomerID'] = df_raw['CustomerID'].astype(int)
        df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])
        df_raw['UnitPrice'] = pd.to_numeric(df_raw['UnitPrice'], errors='coerce')
        df_raw['Quantity'] = pd.to_numeric(df_raw['Quantity'], errors='coerce')
        
        # Remove rows with invalid numeric values
        df_raw.dropna(subset=['UnitPrice', 'Quantity'], inplace=True)
        
        print("\nMissing values after cleaning:")
        print(df_raw.isnull().sum())
        print(f"\nFinal row count: {len(df_raw)}")
        print("\nData types of columns:")
        print(df_raw.dtypes)
        
        print("\nExtraction complete.")
        return df_raw
        
    except FileNotFoundError:
        print(f"Error: Excel file '{data_source}' not found. Please ensure the file is in the correct directory.")
        return None
    except Exception as e:
        print(f"Error during data extraction: {e}")
        return None

Note: you may need to restart the kernel to use updated packages.


In [20]:
# Run the extraction function and display results
import pandas as pd
raw_data = extract()

if raw_data is not None:
    print("\nPreview of the extracted and cleaned data:")
    print("\nFirst 5 rows:")
    print(raw_data.head())
    
    print("\nDataset Info:")
    print(raw_data.info())


--- Starting Extraction Phase ---
Data extracted from C:\Users\flags\End-sem DSA 2040\DSA-2040_Practical_Exam_PatriciaKiarie781\Data Warehousing\ETL_Process\raw\Online Retail.xlsx. Total rows: 541909

Missing values before cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
Data extracted from C:\Users\flags\End-sem DSA 2040\DSA-2040_Practical_Exam_PatriciaKiarie781\Data Warehousing\ETL_Process\raw\Online Retail.xlsx. Total rows: 541909

Missing values before cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


C:\Users\flags\AppData\Local\Temp\ipykernel_9100\2416959743.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_raw['Description'].fillna('Unknown Product', inplace=True)
C:\Users\flags\AppData\Local\Temp\ipykernel_9100\2416959743.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c


Missing values after cleaning:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Final row count: 406829

Data types of columns:
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

Extraction complete.

Preview of the extracted and cleaned data:

First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

  

## Transformation

In [21]:
# Calculate TotalSales column

raw_data['TotalSales'] = raw_data['Quantity'] * raw_data['UnitPrice']

print('Rows after dropna:', len(raw_data))

# Remove outliers: Quantity < 0 or UnitPrice <= 0
raw_data = raw_data[(raw_data['Quantity'] >= 0) & (raw_data['UnitPrice'] > 0)]
print('Rows after removing outliers:', len(raw_data))

# Check min and max InvoiceDate
print('InvoiceDate range:', raw_data['InvoiceDate'].min(), 'to', raw_data['InvoiceDate'].max())

# Try to filter data for sales in the last year (from 2024-08-12 to 2025-08-12)
current_date = pd.Timestamp('2025-08-12')
one_year_ago = current_date - pd.Timedelta(days=365)
df_last_year = raw_data[(raw_data['InvoiceDate'] >= one_year_ago) & (raw_data['InvoiceDate'] <= current_date)]
print('Rows after date filter:', len(df_last_year))

# If no rows, use the most recent year in the data
if df_last_year.empty:
    print('No rows in the last year, using most recent year in data instead.')
    max_date = raw_data['InvoiceDate'].max()
    min_date = max_date - pd.Timedelta(days=365)
    df_last_year = raw_data[(raw_data['InvoiceDate'] >= min_date) & (raw_data['InvoiceDate'] <= max_date)]
    print('Rows in most recent year:', len(df_last_year))

# Create customer summary: total purchases and country
customer_summary = df_last_year.groupby('CustomerID').agg(
    TotalPurchases=('TotalSales', 'sum'),
    Country=('Country', 'first')
).reset_index()

# Display the first few rows of the customer summary
customer_summary.head()

Rows after dropna: 406829
Rows after removing outliers: 397884
InvoiceDate range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Rows after date filter: 0
No rows in the last year, using most recent year in data instead.
Rows in most recent year: 384529


,CustomerID,TotalPurchases,Country
0,12346,77183.60,United Kingdom
1,12347,3598.21,Iceland
2,12348,1797.24,Finland
3,12349,1757.55,Italy
4,12350,334.40,Norway


## Loading

In [22]:
def load(transformed_data, customer_summary, db_name='retail_dw.db'):
    """
    Loads the transformed data into SQLite database with fact and dimension tables
    
    Args:
        transformed_data (pd.DataFrame): The transformed sales data
        customer_summary (pd.DataFrame): The customer dimension data
        db_name (str): Name of the SQLite database file
    """
    print("\n--- Starting Loading Phase ---")
    
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Connected to database: {db_name}")
    
    try:
        # Drop existing tables to ensure a clean load
        cursor.execute("DROP TABLE IF EXISTS CustomerDim;")
        cursor.execute("DROP TABLE IF EXISTS TimeDim;")
        cursor.execute("DROP TABLE IF EXISTS SalesFact;")
        
        # Create CustomerDim table
        cursor.execute("""
            CREATE TABLE CustomerDim (
                customer_id INTEGER PRIMARY KEY,
                total_purchases REAL,
                country TEXT,
                transaction_count INTEGER
            );
        """)
        
        # Create TimeDim table
        cursor.execute("""
            CREATE TABLE TimeDim (
                date_id INTEGER PRIMARY KEY AUTOINCREMENT,
                date DATE,
                year INTEGER,
                month TEXT,
                day_of_week TEXT
            );
        """)
        
        # Create SalesFact table
        cursor.execute("""
            CREATE TABLE SalesFact (
                sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
                invoice_no TEXT,
                customer_id INTEGER,
                date_id INTEGER,
                quantity INTEGER,
                unit_price REAL,
                total_sales REAL,
                FOREIGN KEY(customer_id) REFERENCES CustomerDim(customer_id),
                FOREIGN KEY(date_id) REFERENCES TimeDim(date_id)
            );
        """)
        
        # Prepare and load TimeDim data
        unique_dates = transformed_data['InvoiceDate'].dt.date.unique()
        time_data = []
        for date in unique_dates:
            dt = pd.Timestamp(date)
            time_data.append((
                date,
                dt.year,
                dt.strftime('%B'),
                dt.strftime('%A')
            ))
        
        cursor.executemany(
            "INSERT INTO TimeDim (date, year, month, day_of_week) VALUES (?, ?, ?, ?)",
            time_data
        )
        
        # Load CustomerDim data
        customer_data = customer_summary.to_dict('records')
        cursor.executemany(
            "INSERT INTO CustomerDim (customer_id, total_purchases, country, transaction_count) VALUES (:CustomerID, :total_purchases, :country, :transaction_count)",
            customer_data
        )
        
        # Prepare and load SalesFact data
        # First, get the date_id mapping
        cursor.execute("SELECT date_id, date FROM TimeDim")
        date_mapping = {str(date): id for id, date in cursor.fetchall()}
        
        # Prepare sales fact data
        sales_data = []
        for _, row in transformed_data.iterrows():
            date_id = date_mapping[str(row['InvoiceDate'].date())]
            sales_data.append((
                row['InvoiceNo'],
                int(row['CustomerID']),
                date_id,
                int(row['Quantity']),
                float(row['UnitPrice']),
                float(row['TotalSales'])
            ))
        
        # Insert sales fact data
        cursor.executemany(
            "INSERT INTO SalesFact (invoice_no, customer_id, date_id, quantity, unit_price, total_sales) VALUES (?, ?, ?, ?, ?, ?)",
            sales_data
        )
        
        conn.commit()
        
        # Print summary
        cursor.execute("SELECT COUNT(*) FROM CustomerDim")
        customer_count = cursor.fetchone()[0]
        cursor.execute("SELECT COUNT(*) FROM TimeDim")
        time_count = cursor.fetchone()[0]
        cursor.execute("SELECT COUNT(*) FROM SalesFact")
        sales_count = cursor.fetchone()[0]
        
        print(f"\nLoaded successfully:")
        print(f"- CustomerDim: {customer_count} rows")
        print(f"- TimeDim: {time_count} rows")
        print(f"- SalesFact: {sales_count} rows")
        
    except Exception as e:
        print(f"An error occurred during loading: {e}")
        conn.rollback()
    finally:
        conn.close()
        print("\nDatabase connection closed.")

In [23]:
# Create SQLite database and load data into fact and dimension tables
import os

# Define absolute database file path
db_dir = r'C:\\Users\\flags\\End-sem DSA 2040\\DSA-2040_Practical_Exam_PatriciaKiarie781\\Data Warehousing\\ETL_Process\\raw\\Online Retail.xlsx'
db_path = os.path.join(db_dir, 'C:\\Users\\flags\\End-sem DSA 2040\\DSA-2040_Practical_Exam_PatriciaKiarie781\\Data Warehousing\\ETL_Process\\retail_dw.db')

# Ensure the directory exists
#os.makedirs(db_dir, exist_ok=True)

# Remove existing database for a clean start (optional)
#if os.path.exists(db_path):
    #os.remove(db_path)

conn = sqlite3.connect(db_path)

# Prepare CustomerDim
customer_dim = customer_summary[['CustomerID', 'Country']].drop_duplicates().copy()
customer_dim.to_sql('CustomerDim', conn, index=False, if_exists='replace')

# Prepare TimeDim
time_dim = df_last_year[['InvoiceDate']].drop_duplicates().copy()
time_dim['TimeID'] = time_dim['InvoiceDate'].astype(str)
time_dim['Year'] = time_dim['InvoiceDate'].dt.year
time_dim['Month'] = time_dim['InvoiceDate'].dt.month
time_dim['Day'] = time_dim['InvoiceDate'].dt.day
time_dim.to_sql('TimeDim', conn, index=False, if_exists='replace')

# Prepare SalesFact
sales_fact = df_last_year[['InvoiceNo', 'StockCode', 'CustomerID', 'Quantity', 'UnitPrice', 'TotalSales', 'InvoiceDate']].copy()
sales_fact['TimeID'] = sales_fact['InvoiceDate'].astype(str)
sales_fact.to_sql('SalesFact', conn, index=False, if_exists='replace')

conn.close()
print(f'Data loaded into SQLite database: {db_path}')

Data loaded into SQLite database: C:\Users\flags\End-sem DSA 2040\DSA-2040_Practical_Exam_PatriciaKiarie781\Data Warehousing\ETL_Process\retail_dw.db


In [24]:
# Run the ETL process with detailed logging
run_etl()

Starting ETL process...

1. EXTRACTION

--- Starting Extraction Phase ---
Data extracted from C:\Users\flags\End-sem DSA 2040\DSA-2040_Practical_Exam_PatriciaKiarie781\Data Warehousing\ETL_Process\raw\Online Retail.xlsx. Total rows: 541909

Missing values before cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
Data extracted from C:\Users\flags\End-sem DSA 2040\DSA-2040_Practical_Exam_PatriciaKiarie781\Data Warehousing\ETL_Process\raw\Online Retail.xlsx. Total rows: 541909

Missing values before cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Missing values after cleaning:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0

C:\Users\flags\AppData\Local\Temp\ipykernel_9100\2416959743.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_raw['Description'].fillna('Unknown Product', inplace=True)
C:\Users\flags\AppData\Local\Temp\ipykernel_9100\2416959743.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [25]:
def log_etl_stats(stage, description, row_count, additional_info=None):
    """
    Log statistics for each ETL stage
    
    Args:
        stage (str): ETL stage name (Extract, Transform, or Load)
        description (str): Description of the specific operation
        row_count (int): Number of rows processed
        additional_info (dict, optional): Any additional metrics to log
    """
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"\n[{timestamp}] {stage} - {description}")
    print(f"Rows processed: {row_count:,}")
    
    if additional_info:
        print("Additional metrics:")
        for key, value in additional_info.items():
            print(f"- {key}: {value}")